# SCORE - XGBOOST CLASSIFIER

In [ ]:
import pandas as pd
from sklearn import metrics
import numpy as np
from datetime import date
from sklearn import preprocessing
import xgboost as xgb
import pickle

### GLOBAL VARIABLES

In [ ]:
DATAPATH = '../../../data/features/'
MODELPATH = '../../../models/xgboost/'
SUBMITPATH = '../../../models/submission/'
SEED = 47
NITER = 100
CV = 5
SCORE = 'roc_auc'
handlingnull = False
NJOBS = -1
NTHREADS= 4
USEGPU = False
NCLASS = 0
CUTOFF = 0.5

### LOAD DATASET

In [ ]:
# Load submission Data
submission = pd.read_pickle(DATAPATH + 'submission.pkl')

In [ ]:
submission.shape

In [ ]:
### create a DMatrix and handling Null values
if handlingnull:
    xgsubmit = xgb.DMatrix(submission.values, submission.values, missing=-9999)
else:
    xgsubmit = xgb.DMatrix(submission.values, submission.values)

### LOAD MODEL

In [ ]:
# init model
if USEGPU:
    xgb_model = xgb.Booster({'nthread': NTHREADS, 'tree_method':'gpu_hist', 'gpu_id':0})
else:
    xgb_model = xgb.Booster({'nthread': NTHREADS})  
    
xgb_model.load_model(MODELPATH + 'xgb_XXX.model')  # load data

### SCORING

In [ ]:
y_pred = xgb_model.predict(xgsubmit)

### SUBMIT

In [ ]:
submit = submission.reset_index()

In [ ]:
submit = submit[['prediction_id']]

In [ ]:
submit['prediction'] = y_pred

In [ ]:
submit['class'] = np.where(submit['prediction'] >= CUTOFF, 1, 0)

In [ ]:
submit['prediction'].hist(bins=100)
pass

In [ ]:
submit['class'].hist(bins=100)
pass

In [ ]:
submit.sort_values('prediction_id', ascending=True, inplace=True)

In [ ]:
submit.head()

In [ ]:
submit = submit[['prediction_id', 'class']].sort_values(by='prediction_id')
submit.to_csv(SUBMITPATH + "submit_XXX.csv", index=False, header=True)